In [1]:
from google.colab import files
import zipfile,os,glob,re
import pandas as pd

#機能0：Colab Notebooksにsamplesに入っているファイルをZIP形式で読み込む

#samplesのZIPファイルを選択してアップロード
uploaded = files.upload()

##アップロードしたZipファイルを「unzipped_samples」フォルダの配下に解凍
with zipfile.ZipFile("samples.zip", "r") as zip_ref:
    zip_ref.extractall("unzipped_samples")



Saving samples.zip to samples.zip


In [26]:
# F001: 注文集計機能
# 指定フォルダ内の注文ファイル（order_*.xlsx）を読み込み、注文日ごとに各野菜の合計注文数を集計する

class OrderAggregator:
    def __init__(self, folder_path, file_pattern="order_*.xlsx", sheet_name="Sheet1"):
        self.folder_path  = folder_path     # 注文ファイルのフォルダパス
        self.file_pattern = file_pattern    # ファイル名パターン（例：order_*.xlsx）
        self.sheet_name   = sheet_name      # 読み込むシート名

    def summarize_by_date(self):            # 注文ファイルを読み込み、注文日ごとに商品別注文数を集計して返す
        files   = glob.glob(os.path.join(self.folder_path, self.file_pattern))  # 注文ファイル一覧を取得
        df_list = []
        for f in files:
            match = re.search(r"(\d{8})", f)                                    # ファイル名から注文日（8桁）を抽出
            df    = pd.read_excel(f, sheet_name=self.sheet_name).fillna(0)      # 注文データを読み込み
            df["注文日"] = pd.to_datetime(match.group(1), format="%Y%m%d")      # 注文日を追加
            df_list.append(df)
        return pd.concat(df_list, ignore_index=True).groupby("注文日").sum(numeric_only=True)  # 注文日ごとに集計

aggregator     = OrderAggregator("unzipped_samples/samples/order_new")          # 注文ファイルのフォルダパスを渡して初期化
daily_summary  = aggregator.summarize_by_date()                                 # 注文日ごとの集計結果を取得
print(daily_summary)


             トマト  キャベツ  レタス    白菜  ほうれん草    大根  ニンジン
注文日                                                 
2023-05-24  31.0  21.0   42  25.0   23.0  15.0  32.0


In [27]:
# F002: 在庫確認機能
# 在庫表（inventory.xlsx）の最終行を抽出し、最新の野菜の在庫情報を取得する

class InventoryManager:
    def __init__(self, folder_path, file_name="inventory.xlsx"):
        self.file_path = os.path.join(folder_path, file_name)                   # 在庫ファイルのパス

    def get_latest_inventory(self):                                             # 在庫表の最終行（最新の在庫情報）を取得して返す
        df = pd.read_excel(self.file_path)
        return df.iloc[-1]                                                      # 最終行を返す

manager          = InventoryManager("unzipped_samples/samples")                 # 在庫ファイルのフォルダパスを渡して初期化
latest_inventory = manager.get_latest_inventory()                               # 最新の在庫情報を取得
print(latest_inventory)

日付       2023-05-14 00:00:00
曜日                       Sun
トマト                       91
キャベツ                      73
レタス                      103
白菜                        84
ほうれん草                     75
大根                        48
ニンジン                      50
Name: 13, dtype: object


In [29]:
# F003: しきい値確認機能
# 発注基準となるしきい値と追加量をpickup.xlsxから取得する

class ThresholdLoader:
    def __init__(self, folder_path, file_name="pickup.xlsx"):
        self.file_path = os.path.join(folder_path, file_name)                   # しきい値ファイルのパス

    def load_thresholds(self):                                                  # Excelファイルから「しきい値」「追加量」を辞書形式で取得して返す
        df         = pd.read_excel(self.file_path, index_col=0)
        thresholds = df.loc["しきい値"].to_dict()                               # 各野菜のしきい値を取得
        additions  = df.loc["追加量"].to_dict()                                 # 各野菜の追加量を取得
        return {"しきい値": thresholds, "追加量": additions}

loader = ThresholdLoader("unzipped_samples/samples")                            # しきい値ファイルのフォルダパスを渡して初期化
data   = loader.load_thresholds()                                               # しきい値と追加量を取得
print("しきい値:\n", data["しきい値"])                                          # しきい値を表示
print("追加量:\n", data["追加量"])

しきい値:
 {'トマト': 50, 'キャベツ': 40, 'レタス': 50, '白菜': 30, 'ほうれん草': 40, '大根': 30, 'ニンジン': 40}
追加量:
 {'トマト': 100, 'キャベツ': 80, 'レタス': 100, '白菜': 60, 'ほうれん草': 80, '大根': 60, 'ニンジン': 80}


In [30]:
# F004: 発注判定機能
# 在庫・注文・しきい値をもとに残在庫数を計算し、発注が必要な野菜を抽出する

class PurchasePlanner:
    def __init__(self, inventory_folder, order_folder, threshold_folder):
        self.inventory  = InventoryManager(inventory_folder)                    # 在庫管理クラスを初期化
        self.orders     = OrderAggregator(order_folder)                         # 注文集計クラスを初期化
        self.thresholds = ThresholdLoader(threshold_folder).load_thresholds()   # しきい値データを取得

    def latest_inventory(self):                                                 # 最新の在庫情報を取得
        return self.inventory.get_latest_inventory()

    def total_orders(self):                                                     # 各野菜の合計注文数を取得
        return self.orders.summarize_by_date().sum(numeric_only=True)

    def plan(self):                                                             # 在庫・注文・しきい値・追加量を統合し、残在庫数を計算した一覧を返す
        inv   = self.latest_inventory().drop(labels="日付", errors="ignore")    # 「日付」列を除外（野菜名のみ残す）
        orders= self.total_orders()                                             # 合計注文数を取得
        plan  = pd.DataFrame({"在庫数": inv, "注文合計": orders})               # 在庫数と注文合計を統合
        plan["残在庫数"] = plan["在庫数"] - plan["注文合計"]                    # 残在庫数 = 在庫数 - 注文合計
        plan = plan.drop(index="曜日", errors="ignore")                         # 「曜日」行が存在する場合は除外（不要な行の削除）
        plan["しきい値"] = pd.Series(self.thresholds["しきい値"]).reindex(plan.index)  # 野菜名に合わせてしきい値を結合
        plan["追加量"]   = pd.Series(self.thresholds["追加量"]).reindex(plan.index)   # 野菜名に合わせて追加量を結合
        return plan                                                             # 発注判定に必要な一覧を返す

    def pickup_items(self):                                                     # 残在庫数がしきい値を下回る野菜のみ抽出し、追加量を返す
        plan   = self.plan()
        pickup = plan[plan["残在庫数"] < plan["しきい値"]]                      # 発注対象の野菜を抽出
        return pickup[["追加量"]]                                               # 発注対象の追加量のみ返す

planner = PurchasePlanner("unzipped_samples/samples",
                          "unzipped_samples/samples/order_new",
                          "unzipped_samples/samples")                           # 各ファイルのフォルダパスを渡して初期化

print("全野菜の在庫状況:\n", planner.plan())                                    # 在庫・注文・しきい値・追加量の一覧を表示
print("\n発注対象:\n", planner.pickup_items())


全野菜の在庫状況:
        在庫数  注文合計  残在庫数  しきい値  追加量
ほうれん草   75  23.0  52.0    40   80
キャベツ    73  21.0  52.0    40   80
トマト     91  31.0  60.0    50  100
ニンジン    50  32.0  18.0    40   80
レタス    103  42.0  61.0    50  100
大根      48  15.0  33.0    30   60
白菜      84  25.0  59.0    30   60

発注対象:
       追加量
ニンジン   80
